# Практика 6

# О формате сдачи

🔷 **<font color='plum'>При решении ноутбука используйте данный шаблон</font>**

    ✅ Можно добавлять новые ячейки любых типов
    ❌ Не нужно удалять текстовые ячейки c разметкой частей ноутбука и формулировками заданий


🔷 **<font color='plum'>При оценивании задач учитывается код</font>**

    ✅ Задания, в которых необходим код, обычно помечаются фразами "Your code here"/"Ваш код" и аналогичными
    ❌ Ответы на вопросы без сопутствующего кода оцениваются в 0 баллов
    ❌ Наличе работоспособного кода в ноутбуке, если на сказано иного, обязательно

🔷 **<font color='plum'>При оценивании задач учитываются выводы</font>**

    ✅ Задания, в которых необходимы выводы, обычно помечаются фразами Вывод"/"Ответ на вопрос"/"Ваш текст" и аналогичными
    ✅ Обычно выводы подразумевают под собой текстовый ответ (можно писать markdown, latex).
    ✅ Сопутствующие изображения, графики, таблички - приветствуются!
    ❌ При отсутствии выводов задание не засчитается на полный балл

In [3]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

import plotly.io as pio
import matplotlib_inline

sns.set(style="ticks", context="talk")
plt.style.use("dark_background")
pio.templates.default = pio.templates.merge_templates("plotly_dark", "presentation")
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf', 'svg')

pd.set_option('display.max_columns', None)

# Кластеризация иерархии животных



Будем использовать данные, описывающие признаки различных животных https://www.kaggle.com/datasets/uciml/zoo-animal-classification/data

<font color='CornflowerBlue'>**Цель**</font> --- выявить кластера различных классов животных, а также построить их иерархию

In [4]:
! gdown 106wQ0gOlT8drsG0OeWBAM_s7CgEsUxVc
! gdown 1j9o40Aubng-Hoe9kaGwSIXD6s9z_7e8c

Downloading...
From: https://drive.google.com/uc?id=106wQ0gOlT8drsG0OeWBAM_s7CgEsUxVc
To: /content/zoo.csv
100% 4.37k/4.37k [00:00<00:00, 8.25MB/s]
Downloading...
From: https://drive.google.com/uc?id=1j9o40Aubng-Hoe9kaGwSIXD6s9z_7e8c
To: /content/class.csv
100% 963/963 [00:00<00:00, 3.21MB/s]


In [5]:
data = pd.read_csv("/content/zoo.csv")
classes = pd.read_csv("/content/class.csv")

Посмотрим на данные

In [6]:
data.head()

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1


In [7]:
classes.head(n=10)

,Class_Number,Number_Of_Animal_Species_In_Class,Class_Type,Animal_Names
0,1,41,Mammal,"aardvark, antelope, bear, boar, buffalo, calf,..."
1,2,20,Bird,"chicken, crow, dove, duck, flamingo, gull, haw..."
2,3,5,Reptile,"pitviper, seasnake, slowworm, tortoise, tuatara"
3,4,13,Fish,"bass, carp, catfish, chub, dogfish, haddock, h..."
4,5,4,Amphibian,"frog, frog, newt, toad"
5,6,8,Bug,"flea, gnat, honeybee, housefly, ladybird, moth..."
6,7,10,Invertebrate,"clam, crab, crayfish, lobster, octopus, scorpi..."


Всего <font color='CornflowerBlue'>**7 классов**</font> животных. Сначала мы посмотрим на кластеризацию, а затем будем сравнивать ее истиной разметкой

In [ ]:
names = data["animal_name"]
data.drop("animal_name", axis=1, inplace=True)

y = data["class_type"]
data.drop("class_type", axis=1, inplace=True)

### **<font color='DarkOrange'> Задача 1.**</font>

#### <font color='plum'>**Исследование признаков**</font>

* Исследуйте признаки в данных `data`. Какого они типа (в классификации вещественный/категориальный/бинарный итд)?
* Отметьте выделяющиеся признаки и преобразуйте их в такой формат, как и большинство остальных признаков

In [ ]:
data = # Your code here

### **<font color='DarkOrange'> Задача 2.**</font>
#### <font color='plum'>**Масштабирование**</font>

Мы хотим применить алгоритм кластеризации для выявления похожих кластеров. Нужно ли в данном случае (на преобразованных выше данных) приводить признаки в одному масштабу? Почему?

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

------------

Далее будем оценивать качество нашей кластеризации как по метрикам, так и по визуальной оценке. Для этого приведем вспомогательную функцию. Если хотите что-то в ней улучшить/поменять - это можно сделать

In [ ]:
from functools import partial

def show_slider(X_reduced, cluster_colors, iteration, title, prefix, suffix=""):
    """
    X_reduced: 2d np.array or pd.DataFrame, исходные данные, у которых снизили размерность
    по признакам до 2 или 3 фичей

    cluster_colors: list[iterable] - список раскрасок (кластеров), соответствующих
    каждому перебранному параметру K из iteration. Каждый элемент этого списка это фактически
    predict алгоритма кластеризации

    iteration: list[int] - сетка перебора параметра K

    title: str - название графика

    prefix: str - префикс названия слайдера

    suffix: str - суффикт названия слайдера

    """

    fig = go.Figure()

    method_2d = partial(px.scatter, x=0, y=1, color='cluster')
    method_3d = partial(px.scatter_3d, x=0, y=1, z=2, color='cluster')

    method = method_2d if X_reduced.shape[1] == 2 else method_3d

    for y in cluster_colors:
        df = pd.DataFrame(X_reduced)
        df["cluster"] = y.astype(int)
        fig.add_traces(list(method(data_frame=df).select_traces()))

    for n in range(1, len(fig.data)):
        fig.data[n].visible = False
    fig.data[0].visible = True

    steps = []
    for i in range(len(fig.data)):
        step = dict(
            method='update',
            args=[
                {'visible': [False] * len(fig.data)},
                {'color_continuous_scale': [px.colors.sequential.Plasma] * len(fig.data)},
                {'title.text': [title] * len(fig.data)}
            ],
            label=str(iteration[i]),
        )
        step['args'][0]['visible'][i] = True
        steps.append(step)

    sliders = [dict(
        active=0,
        steps=steps,
        currentvalue={"prefix": prefix, "suffix": suffix},
    )]

    fig.update_yaxes(autorange='reversed', scaleanchor='x', constrain='domain')
    fig.update_xaxes(constrain='domain')

    fig.layout.sliders = sliders
    fig.layout.title = title

    fig.show()

### **<font color='DarkOrange'> Задача 3.**</font>

#### <font color='plum'>**K-means**</font>

* Обучите несколько вариантов `k-means` с разными `k`
    * На основе имеющихся данных, какую бы сетку для перебора `k` (количество кластеров) вы сделали бы и почему?


In [ ]:
from sklearn.cluster import KMeans
import tqdm

# your code here

cluster_colors = []
K = # <your code here>

for k in tqdm.tqdm(K):
    # your code here
    # cluster_colors.append(kmeans.predict(...)) - как заполнять cluster_colors

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

---------

### **<font color='DarkOrange'> Задача 4.**</font>

#### <font color='plum'>**Понижение размерности**</font>

* Сгенерируйте 2 разных варианта понижения размерности (для 2d случая и 3d случая) для каждого из следующих алгоритмов:
    * <font color='CornflowerBlue'>**PCA**</font>
    * <font color='CornflowerBlue'>**TSNE**</font> (`perplexity` можно выбрать в районе 6-8)
    * <font color='CornflowerBlue'>**Isomap**</font> (`n_neighbors` можно выбрать в районе 20-30)

Итого у вас должно получиться <font color='CornflowerBlue'>**6 разных вариантов**</font> понижения размерности

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE, Isomap

In [ ]:
# your code here

### **<font color='DarkOrange'> Задача 5.**</font>

#### <font color='plum'>**Поиск наилучшей кластеризации**</font>

* Найдите оптимальное число кластеров используя
    *  <font color='CornflowerBlue'>**оценку визуально**</font> - посмотрите на все 6 вариантов понижения размерности со слайдером по разным `k`
        *  Какие методы понижения размерности оказались хорошими для визуализации кластеров?
    *  <font color='CornflowerBlue'>**silhouette_score**</font> - постройте график зависимости этой метрики от `k`


*  Какое оптимальное число кластеров получается для обоих способов их поиска? Почему вы остановились именно на таких значениях?

In [ ]:
from sklearn.metrics import silhouette_score

# your code here

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

### **<font color='MediumSeaGreen'> Бонус 1  </font>**

* Найдите также значение оптимального $k$ [методом локтя](https://www.geeksforgeeks.org/elbow-method-for-optimal-value-of-k-in-kmeans/), используя параметр [intertia_](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) класса Kmeans. Не забудьте построить график зависимости `inertia_` от `k`. Изучите этот метод и инерцию при необходимости самостоятельно.
* Совпал ли ваш выбор с предыдущим заданием?

In [ ]:
# your code here

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

### **<font color='MediumSeaGreen'> Бонус 2  </font>**

В самом датасете также присутствует и истинная разметка данных по классам.

* Изучите, как устроена внешняя [метрика качества BCubed](https://habr.com/ru/companies/yandex/articles/500742/), найдите реализацию или напишите ее сами.
* Выберите наилучший `k` исходя из имеющихся данных об эталонной разметке используя метрику `BCubed`
* Совпадает ли ваш выбор с предыдущими заданиями?

In [ ]:
# your code here

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

-----------

### Иерархическая кластеризация

Так как само разбиение животных по классам подразумевает некую иерархию между видами, то посмотрим, как с поиском иерархии справляется <font color='CornflowerBlue'>**агломеративная кластеризация.**</font>

Для более красивой визуализации мы можем обойтись без `sklearn` - только средствами библиотек `scipy` и `plotly`

In [ ]:
import plotly.figure_factory as ff
from functools import partial
from scipy.cluster.hierarchy import linkage

In [ ]:
def show_slider_dendrogram(figures, ths, title, prefix, suffix=""):
    fig = go.Figure()

    for f in figures:
        fig.add_traces(list(f.select_traces()))

    batch_data = len(fig.data) // len(figures)

    for n in range(len(fig.data)):
        if n < batch_data:
            fig.data[n].visible = True
        else:
            fig.data[n].visible = False


    steps = []
    for i in range(len(figures)):
        step = dict(
            method='update',
            args=[
                {'visible': [False] * len(fig.data)},
            ],
            label=str(ths[i]),
        )
        for j in range(i * batch_data, (i + 1) * batch_data):
            step['args'][0]['visible'][j] = True
        steps.append(step)

    sliders = [dict(
        active=0,
        steps=steps,
        currentvalue={"prefix": prefix, "suffix": suffix},
    )]

    fig.update_yaxes(constrain='domain')
    fig.update_xaxes(constrain='domain')

    fig.update_layout(
        xaxis = dict(
            mirror='allticks',
            rangemode='tozero',
            showticklabels=True,
            ticks='outside',
            tickmode='array',
            tickvals = list(figures[0].select_xaxes())[0].tickvals,
            ticktext = list(figures[0].select_xaxes())[0].ticktext,
            type='linear',
            zeroline=False
        )
    )

    fig.layout.sliders = sliders
    fig.layout.title = title

    fig['layout']['sliders'][0]['pad']=dict(r= 10, t= 200,)

    fig.update_layout(height=700, width=1800)

    fig.show()

Соединим названия животных с их классом

In [ ]:
animal_class_names = list(classes["Class_Type"][y - 1])
merged_names = pd.DataFrame()
merged_names["animal_class"] = animal_class_names
merged_names["animal_name"] = np.array(names)
merged_names["merge"] = merged_names["animal_name"] + " (" +  merged_names["animal_class"] + ")"

In [ ]:
merged_names

,animal_class,animal_name,merge
0,Mammal,aardvark,aardvark (Mammal)
1,Mammal,antelope,antelope (Mammal)
2,Fish,bass,bass (Fish)
3,Mammal,bear,bear (Mammal)
4,Mammal,boar,boar (Mammal)
...,...,...,...
96,Mammal,wallaby,wallaby (Mammal)
97,Bug,wasp,wasp (Bug)
98,Mammal,wolf,wolf (Mammal)
99,Invertebrate,worm,worm (Invertebrate)


### **<font color='DarkOrange'> Задача 6.**</font>

#### <font color='plum'>**Поиск наилучшей иерархии**</font>

* Вместо аггломеративной кластеризации в `sklearn` будем использовать аналог из `plotly`. Изучите метод [ff.create_dendrogram](https://plotly.com/python/dendrogram/), в частности за что отвечает параметр `color_threshold`.

* Для каждого из <font color='CornflowerBlue'>**3 типов связности**</font> (`ward`, `single`, `complete`) подберите свою сетку перебора параметра отнесения объектов к одному кластеру (`color_threshold`). Понять оптимальную сетку можно также поигравшись визуально

* Найдите оптимальное число кластеров используя <font color='CornflowerBlue'>**оценку визуально**</font>

In [ ]:
THRESHOLDS = {
    #"ward": np.linspace(...), # your code here
    #"single": np.linspace(...),
    #"complete": np.inspace(...)
}

for link in ['ward', 'single', 'complete']:
    figures = []
    for th in THRESHOLDS[link]:
        figures.append(ff.create_dendrogram(data, color_threshold=th,
                            linkagefun=lambda x: linkage(data, link, metric='euclidean'),
                            labels=np.array(merged_names["merge"])))
    show_slider_dendrogram(figures, THRESHOLDS[link], f"Linkage={link}", "Color threshold= ", "")

### **<font color='orange'> Задача 7.**</font>

#### <font color='plum'>**Анализ иерархии**</font>

* Совпадает ли оптимальное число кластеров с выбранными методом `k-means?`
* Какой из типов связей `(linkage)` оказался наилучшим?
* Какие животные кластеризовались неправильно?
* Почему некоторые классы визуально содержат большие группы подклассов? Что это за классы и по каким признаками можно выделить эти подгруппы?

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

# Кластеризация точек WiFi



Будем использовать данные о [точках городской сети WiFi](https://data.mos.ru/opendata/2756/description?isDynamic=false&version=1&release=33)

In [ ]:
! gdown 1e4lgJL3RNMLAPdnZVg5DgbV_nNg7lAYE

Downloading...
From: https://drive.google.com/uc?id=1e4lgJL3RNMLAPdnZVg5DgbV_nNg7lAYE
To: /content/data-2756-2023-09-22.csv
100% 1.39M/1.39M [00:00<00:00, 114MB/s]


In [ ]:
wifi = pd.read_csv("/content/data-2756-2023-09-22.csv", sep=';')

In [ ]:
wifi.head()

,Latitude_WGS84,Name,AdmArea,District,Location,NumberOfAccessPoints,WiFiName,CoverageArea,FunctionFlag,AccessFlag,Password,Longitude_WGS84,global_id,geoData,geodata_center,Unnamed: 15
0,Широта в WGS-84,Наименование,Административный округ,Район,Месторасположение,Количество точек доступа,Имя Wi-Fi сети,Зона покрытия (метры),Признак функционирования,Условия доступа,Пароль,Долгота в WGS-84,global_id,geoData,geodata_center,NaN
1,55.767529404,Городской Wi-Fi,Центральный административный округ,Басманный район,"город Москва, Старая Басманная улица, дом 22",1,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,37.666264797,276298117,"{coordinates=[37.666264797, 55.767529404], typ...","{coordinates=[37.666264797, 55.767529404], typ...",NaN
2,55.767096294,Городской Wi-Fi,Центральный административный округ,Басманный район,"город Москва, Старая Басманная улица, дом 23/9...",2,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,37.664446511,276298119,"{coordinates=[37.664446511, 55.767096294], typ...","{coordinates=[37.664446511, 55.767096294], typ...",NaN
3,55.768620654,Городской Wi-Fi,Центральный административный округ,Басманный район,"город Москва, Старая Басманная улица, дом 33",2,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,37.667454947,276298121,"{coordinates=[37.667454947, 55.768620654], typ...","{coordinates=[37.667454947, 55.768620654], typ...",NaN
4,55.768975817,Городской Wi-Fi,Центральный административный округ,Басманный район,"город Москва, Старая Басманная улица, дом 34",2,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,37.668718073,276298122,"{coordinates=[37.668718073, 55.768975817], typ...","{coordinates=[37.668718073, 55.768975817], typ...",NaN


In [ ]:
wifi.drop("Unnamed: 15", axis=1, inplace=True)
wifi.drop(0, axis=0, inplace=True)

In [ ]:
wifi.Longitude_WGS84 = wifi.Longitude_WGS84.astype(float)
wifi.Latitude_WGS84 = wifi.Latitude_WGS84.astype(float)

Заметим, что у каждой точки есть количество точек подключения. Давайте продублируем каждый объект столько раз, сколько точек подключения у него есть

In [ ]:
wifi2 = wifi.loc[wifi.index.repeat(wifi.NumberOfAccessPoints)]
wifi2.head()

,Latitude_WGS84,Name,AdmArea,District,Location,NumberOfAccessPoints,WiFiName,CoverageArea,FunctionFlag,AccessFlag,Password,Longitude_WGS84,global_id,geoData,geodata_center
1,55.767529,Городской Wi-Fi,Центральный административный округ,Басманный район,"город Москва, Старая Басманная улица, дом 22",1,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,37.666265,276298117,"{coordinates=[37.666264797, 55.767529404], typ...","{coordinates=[37.666264797, 55.767529404], typ..."
2,55.767096,Городской Wi-Fi,Центральный административный округ,Басманный район,"город Москва, Старая Басманная улица, дом 23/9...",2,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,37.664447,276298119,"{coordinates=[37.664446511, 55.767096294], typ...","{coordinates=[37.664446511, 55.767096294], typ..."
2,55.767096,Городской Wi-Fi,Центральный административный округ,Басманный район,"город Москва, Старая Басманная улица, дом 23/9...",2,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,37.664447,276298119,"{coordinates=[37.664446511, 55.767096294], typ...","{coordinates=[37.664446511, 55.767096294], typ..."
3,55.768621,Городской Wi-Fi,Центральный административный округ,Басманный район,"город Москва, Старая Басманная улица, дом 33",2,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,37.667455,276298121,"{coordinates=[37.667454947, 55.768620654], typ...","{coordinates=[37.667454947, 55.768620654], typ..."
3,55.768621,Городской Wi-Fi,Центральный административный округ,Басманный район,"город Москва, Старая Басманная улица, дом 33",2,Moscow_WiFi_Free,50,действует,открытая сеть,NaN,37.667455,276298121,"{coordinates=[37.667454947, 55.768620654], typ...","{coordinates=[37.667454947, 55.768620654], typ..."


In [ ]:
fig = px.scatter_mapbox(wifi2,
                        lat='Latitude_WGS84',
                        lon='Longitude_WGS84',
                        text='NumberOfAccessPoints',
                        zoom=8)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(height=1000, width=1500)
fig.show()

### **<font color='DarkOrange'> Задача 8.**</font>

#### <font color='plum'>**Поиск точек с наилучшим доступом к WiFi**</font>

* Используя метод <font color='CornflowerBlue'>**DBSCAN**</font> найдите те точки, в которых WiFi представлен наилучшем образом.
    * Поймите, какой порядок `eps` стоит взять в случае точек доступа к WiFi.
    * Поиграйтесь с разными `eps` и `min_samples`

* При каких параметрах выделяется на всю Москву достаточно малое (скажем порядка `30`) количество точек с наилучшим доступом? Что это за места? Какие места выделились вне Садового кольца?

In [ ]:
from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=#your code
                min_samples=#your code
)

dbscan.fit(wifi2[["Latitude_WGS84", "Longitude_WGS84"]])

colors = dbscan.labels_ # -1 это шумовые объекты
colors[colors != -1] = 1
colors[colors == -1] = 0

wifi2["colors"] = colors

In [ ]:
fig = px.scatter_mapbox(wifi2[wifi2.colors == 0],
                        lat='Latitude_WGS84',
                        lon='Longitude_WGS84',
                        text='NumberOfAccessPoints',
                        zoom=8)

fig2 = px.scatter_mapbox(wifi2[wifi2.colors == 1],
                        lat='Latitude_WGS84',
                        lon='Longitude_WGS84',
                        text='NumberOfAccessPoints',
                        zoom=8)
fig2.update_traces(cluster=dict(enabled=True, step=3, opacity=1))


fig.add_trace(fig2._data[0])
fig.data[0].marker.color = 'rgba(255,0,0,1)'


fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(height=1000, width=1500)
fig.show()

<font color='MediumOrchid'>**Ваши выводы тут:**</font>

### **<font color='MediumSeaGreen'> Бонус </font>**

* Возьмите любые данные, связанные с геопозицией (например, можно найти на том же сайте https://data.mos.ru/ или https://www.datacatalogs.ru). Можно найти где-нибудь еще
* Для этих данных придумайте задачу/гипотезу, в решении/проверке которой бы помогло решение задачи кластеризации на геопозиции объектов.
    * например, как мы делали на семинаре: предположили, что в торговых центрах больше скопления точек с едой, и таким образом нашли торговые центы

* Решите поставленную задачу методами кластеризации, отобразите результат на карте

* Получилось ли решить поставленную задачу/гипотезу? Если нет, то как вы думаете, почему?

In [ ]:
#your code

<font color='MediumOrchid'>**Ваши выводы тут:**</font>